在某些业务场景下，我们需要不同的分支来完成任务，例如根据用户输入的问题类型，选择不同的分支来处理，或者有条件地终止执行。LangGraph提供了条件边add_conditional_edges来帮助我们实现这个场景，同时配合Send和Command能够完成更高阶的操作，下面我们先来介绍条件边的创建方法。

# add_conditional_edges

dd_conditional_edges方法用于向StateGraph添加条件边，允许根据条件动态决定下一个要执行的节点，该方法接收以下参数：
  - source: 起始节点名称，条件边会在该节点执行完毕后触发
  - path: 路由函数或Runnable，用于根据当前状态确定下一个节点
    - 如果不指定 path_map，应直接返回节点名称或节点名称列表
    - 如果返回 END，则图执行停止
  - path_map: 可选的返回值到节点名的映射字典
    - 如果省略，则 path 函数应直接返回节点名称

## 路由函数直接返回节点名称

当路由函数直接返回图中已存在的节点名时，可以省略第三个映射字典参数。

In [5]:
from traceback import print_tb
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class State(TypedDict):
    query: str
    response: str

# 路由函数直接返回节点名
def route_directly(state: State) -> str:
    if "math" in state["query"].lower():
        return "math_node"   # ← 直接返回节点名
    else:
        return "general_node" # ← 直接返回节点名

def math_node(state: State):
    return {"response": "🔢 我会算术！"}

def general_node(state: State):
    return {"response": "💬 我是一个通用助手。"}

# 构建图
builder = StateGraph(State)
builder.add_node("math_node", math_node)
builder.add_node("general_node", general_node)

# 关键：只传两个参数！
builder.add_conditional_edges(
    START,
    route_directly
    # 没有第三个参数
)

builder.add_edge("math_node", END)
builder.add_edge("general_node", END)
graph = builder.compile()

# 测试
response = graph.invoke({"query": "你好"})
print(response["response"])
# 输出: 🔢 我会算术！

💬 我是一个通用助手。


## 路由函数返回逻辑标签

当你的路由函数返回的是业务语义标签（而非具体节点名）时，必须提供映射字典。这是推荐的做法，因为通过路由逻辑（意图识别）和执行逻辑（节点实现）完全分离解耦，它可以使你的代码更具可读性和可维护性，搭配Literal，能防止拼写错误，提供更加健壮的代码。

In [6]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

class State(TypedDict):
    query: str
    response: str

# 路由函数返回语义标签（与节点名解耦）
def route_by_intent(state: State) -> Literal["calculation", "chat"]:
    if any(word in state["query"].lower() for word in ["+", "-", "*", "/"]):
        return "calculation"  # ← 返回业务意图
    else:
        return "chat"         # ← 返回业务意图

def calculator(state: State):  # 注意：节点名叫 calculator，不是 calculation
    return {"response": "🧮 正在计算..."}

def chatbot(state: State):     # 节点名叫 chatbot，不是 chat
    return {"response": "🗣️ 正在聊天..."}

# 构建图
builder = StateGraph(State)
builder.add_node("calculator", calculator)
builder.add_node("chatbot", chatbot)

# 关键：提供第三个参数——映射字典
builder.add_conditional_edges(
    START,
    route_by_intent,
    {
        "calculation": "calculator",  # 语义标签(函数返回的) → 节点名
        "chat": "chatbot"
    }
)

builder.add_edge("calculator", END)
builder.add_edge("chatbot", END)
graph = builder.compile()

# 测试
print(graph.invoke({"query": "What is 10 * 5?"})["response"])
# 输出: 🧮 正在计算...

🧮 正在计算...
